In [33]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [34]:
import pandas as pd

movies = pd.read_csv("data/movies.csv",encoding="Latin1")
Ratings = pd.read_csv("data/ratings.csv")
Tags = pd.read_csv("data/tags.csv",encoding="Latin1")

Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()


,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,1,1,4.0,964982703,4.366379,-0.366379
1,1,3,4.0,964981247,4.366379,-0.366379
2,1,6,4.0,964982224,4.366379,-0.366379
3,1,47,5.0,964983815,4.366379,0.633621
4,1,50,5.0,964982931,4.366379,0.633621


In [35]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [36]:
Ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [37]:
Tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [38]:
check = pd.pivot_table(Rating_avg,values = 'rating_x',index='userId',columns='movieId')
check.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId')
final.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,NaN,-0.366379,NaN,NaN,-0.366379,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
#Replacing NaN by Movie Average
final_movie = final.fillna(final.mean(axis=0))

#Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()),axis=1)

In [41]:
final_movie.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,-0.053158,-0.366379,-1.096045,-0.522626,-0.366379,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024
2,0.312167,-0.053158,-0.234798,-1.096045,-0.522626,0.378461,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024
3,0.312167,-0.053158,-0.234798,-1.096045,-0.522626,0.378461,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024
4,0.312167,-0.053158,-0.234798,-1.096045,-0.522626,0.378461,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024
5,0.363636,-0.053158,-0.234798,-1.096045,-0.522626,0.378461,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024


In [42]:
final_user.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-3.663793e-01,1.837611e-16,-3.663793e-01,1.837611e-16,1.837611e-16,-3.663793e-01,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,...,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16
2,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,...,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16
3,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,...,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16
4,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,...,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16
5,3.636364e-01,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,...,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16


In [43]:
#user similarity on replacing NaN by user avg
b=cosine_similarity(final_user)
np.fill_diagonal(b,0)
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,1.264516e-03,5.525772e-04,0.048419,0.021847,-0.045497,-6.199672e-03,0.047013,1.950985e-02,-8.754088e-03,...,0.018127,-0.017172,-0.015221,-3.705875e-02,-2.912138e-02,0.012016,0.055261,0.075224,-2.571255e-02,0.010932
2,0.001265,0.000000e+00,4.336877e-30,-0.017164,0.021796,-0.021051,-1.111357e-02,-0.048085,1.020227e-29,3.011629e-03,...,-0.050551,-0.031581,-0.001688,1.168436e-30,-4.438744e-30,0.006226,-0.020504,-0.006001,-6.009100e-02,0.024999
3,0.000553,4.336877e-30,0.000000e+00,-0.011260,-0.031539,0.004800,4.870809e-31,-0.032471,2.134412e-30,8.283549e-31,...,-0.004904,-0.016117,0.017749,2.724547e-31,-1.430628e-03,-0.037289,-0.007789,-0.013001,4.841384e-30,0.019550
4,0.048419,-1.716402e-02,-1.125978e-02,0.000000,-0.029620,0.013956,5.809139e-02,0.002065,-5.873603e-03,5.159032e-02,...,-0.037687,0.063122,0.027640,-1.378212e-02,4.003747e-02,0.020590,0.014628,-0.037569,-1.788358e-02,-0.000995
5,0.021847,2.179571e-02,-3.153892e-02,-0.029620,0.000000,0.009111,1.011715e-02,-0.012284,2.710075e-30,-3.316512e-02,...,0.015964,0.012427,0.027076,1.246135e-02,-3.627206e-02,0.026319,0.031896,-0.001751,9.382892e-02,-0.000278


In [44]:
#user similarity on replacing NaN by item(movie) avg
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine,0)
similarity_with_movie=pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.988283,0.978406,0.964220,0.986819,0.970456,0.971643,0.987468,0.986382,0.973397,...,0.987335,0.978916,0.917922,0.983978,0.978638,0.959693,0.976120,0.932806,0.989380,0.952774
2,0.988283,0.000000,0.987141,0.971166,0.995793,0.979893,0.981852,0.995168,0.995108,0.981285,...,0.996067,0.988455,0.929086,0.993014,0.988206,0.968868,0.983619,0.940224,0.997957,0.963114
3,0.978406,0.987141,0.000000,0.961237,0.985179,0.970773,0.971932,0.985140,0.985263,0.971464,...,0.986072,0.978562,0.921433,0.983193,0.978363,0.957067,0.974114,0.930653,0.988086,0.954265
4,0.964220,0.971166,0.961237,0.000000,0.968638,0.955187,0.958876,0.970090,0.969158,0.959626,...,0.970625,0.964815,0.903118,0.967106,0.963962,0.942701,0.958891,0.911591,0.972100,0.935866
5,0.986819,0.995793,0.985179,0.968638,0.000000,0.978368,0.980011,0.992905,0.993494,0.979161,...,0.994448,0.986028,0.928126,0.991066,0.986090,0.967230,0.982366,0.938353,0.996584,0.960854


In [45]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values,axis=1)[:,:n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:n].index,index=['top{}'.format(i) for i in range(1,n+1)]),axis=1)
    return df
    

In [46]:
#top 30 neighbours for each user
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
1,301,597,414,477,57,369,206,535,590,418,...,484,469,72,593,44,297,434,483,449,552
2,189,246,378,209,227,326,393,332,196,528,...,114,153,596,495,407,567,93,308,496,222
3,441,496,549,231,527,537,313,518,244,246,...,309,586,230,303,34,161,39,47,127,138
4,75,137,590,391,43,128,462,250,290,85,...,472,593,299,32,349,268,159,199,173,426
5,145,35,565,134,58,444,446,347,530,142,...,94,569,411,588,584,404,498,323,536,437


In [47]:
#top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
1,49,54,72,53,515,550,189,133,26,513,...,145,60,595,574,556,92,435,163,439,340
2,49,189,515,53,25,145,54,26,87,194,...,65,163,251,150,299,440,519,507,245,521
3,515,49,25,53,496,54,442,72,26,87,...,92,595,556,65,463,251,180,150,163,439
4,581,54,189,49,25,515,300,53,472,251,...,26,289,463,609,214,206,507,574,120,556
5,145,49,515,53,609,26,189,550,35,87,...,293,142,72,92,556,439,478,163,595,299


In [48]:
def get_user_similar_movies(user1,user2):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(Rating_avg[Rating_avg.userId == user2], on="movieId",how="inner")
    return common_movies.merge(movies,on='movieId')

In [49]:
a = get_user_similar_movies(370,86309)
a = a.loc[:,['rating_x_x','rating_x_y','title']]
a.head()

,rating_x_x,rating_x_y,title


In [50]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [51]:
score = User_item_score(320,7371)
print("score (u,i) is",score)

score (u,i) is 4.071266730936566


In [52]:
Rating_avg = Rating_avg.astype({"movieId": str})
Movie_user = Rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))

In [53]:
def User_item_score1(user,n):

    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'movieId':Movies_under_consideration,'score':score})
    top_recommendation = data.sort_values(by='score',ascending=False).head(n)
    Movie_Name = top_recommendation.merge(movies, how='inner', on='movieId')
    # Movie_Names = Movie_Name.title.values.tolist()

    links = pd.read_csv("data/links.csv",header=0,delim_whitespace=False)
    new_df = links.loc[links['movieId'].isin(Movie_Name['movieId'])]
    df_f = pd.merge(Movie_Name,new_df)

    poster = []
    api_key = "40f84ea0b622bc4257dabf9631a401dc"
    api_version = 3
    api_base_url = f"https://api.themoviedb.org/{api_version}"
    for movie_id in df_f['tmdbId']:    
        endpoint_path = f"/movie/{movie_id}"
        endpoint = f"{api_base_url}{endpoint_path}?api_key={api_key}&language=en-US"
        data = requests.get(endpoint)
        data = data.json()
        poster_path = data['poster_path']
        full_path = "https://image.tmdb.org/t/p/w500/"+poster_path
        poster.append(full_path)

    df_f['Poster'] = poster
    
    js = df_f.to_json(orient='index')

    final_list = list()
    for row in df_f.iterrows():
        final_list.append(row[1].to_dict())
    
    with open('notebooks/json_data/data_user.json', 'w') as f:
        f.write(json.dumps(final_list, indent=4))

    # with open('json_data/data.json', 'w', encoding='utf-8') as f:
    #     json.dump(js, f, ensure_ascii=False, indent=4)
    print(json.dumps(final_list, indent=4))
    

    return df_f






In [54]:
User_item_score1(5,10)

ConnectionError: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/movie/359940.0?api_key=40f84ea0b622bc4257dabf9631a401dc&language=en-US (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000025D9EC605C0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))

In [ ]:
user = int(input("Enter the user id to whom you want to recommend : "))
n=10
predicted_movies = User_item_score1(user,n)
print(" ")
print("The Recommendations for User Id :", user)
print("   ")
for i in predicted_movies:
    print(i)

 
The Recommendations for User Id : 5
   
Three Billboards Outside Ebbing, Missouri (2017)
Return of Martin Guerre, The (Retour de Martin Guerre, Le) (1982)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
Godfather, The (1972)
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
Apocalypse Now (1979)
Boogie Nights (1997)
Monty Python and the Holy Grail (1975)
Casablanca (1942)
Brazil (1985)
